# Reflexer: Validation Model & Simulation

In [1]:
import matplotlib.pyplot as plt
from FixedPoint import FXnum
from decimal import Decimal
import itertools
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
pd.options.plotting.backend = "plotly"

import sys
sys.path.append('./models')

from models.run import run
import models.options as options

## Debt Price Data Source

In [2]:
from models.utils.load_data import debt_price_data

debt_price_source = options.DebtPriceSource.EXTERNAL.value
# options.DebtPriceSource.DEBT_MARKET_MODEL.value
# debt_price_source = options.DebtPriceSource.DEFAULT.value

debt_price_data = debt_price_data(debt_price_source)

/Users/bscholtz/workspace/reflexer/venv/lib/python3.8/site-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


[{'kind': 'drive#file', 'id': '1oESvLtgVD7Ww2LLf4qAT8jtZcjWJjV9KpMN5frP53Zk', 'name': 'debt-price-test-data', 'mimeType': 'application/vnd.google-apps.spreadsheet'}]


In [3]:
debt_price_data[0].head()

,seconds_passed,price_move
0,0,0
1,5185,0
2,4571,0
3,4527,0
4,10071,0


## PI Controller Sweep

In [4]:
from models.utils.load_data import step_dataframe

# debt_price_data = [step_dataframe(20, i) for i in np.around(np.arange(0.05, 1.0, 0.05), decimals=2)]
debt_price_data = [step_dataframe(20, 0.1)]

In [5]:
debt_price_data[0].iloc[20]

seconds_passed    3600.0
price_move           0.1
Name: 20, dtype: float64

### PI Tuning

In [6]:
kp = [6.944e-06]
ki = [60.0/(24*3600)]

In [7]:
controller_sweep = list(itertools.product(kp, ki))
controller_sweep

[(6.944e-06, 0.016666666666666666)]

In [8]:
kp_sweep = [x[0] for x in controller_sweep]
ki_sweep = [x[1] for x in controller_sweep]

## Simulation Configuration

Set the simulation timesteps to the minimum dataset length:

In [9]:
minimum_timesteps = min([df.shape[0] for df in debt_price_data])
SIMULATION_TIMESTEPS = range(minimum_timesteps)
SIMULATION_TIMESTEPS

range(0, 720)

Override certain parameter sweeps with the generated sets, using ConfigWrapper for convenience:

In [10]:
from models.config_wrapper import ConfigWrapper

import models.market_model as market_model
import models.controller_model as controller_model

# Update parameter options
update_params = {
    # By using an Enum, we can self-document all possible options
    options.DebtPriceSource.__name__: [debt_price_source],
    options.IntegralType.__name__: [options.IntegralType.LEAKY.value],
    # Generate a lambda for each test dataframe, that returns a row value at a specific timestep
    'seconds_passed': [
        lambda timestep, df=df.copy(): int(df.iloc[timestep - 1]['seconds_passed'])
        for df in debt_price_data
    ],
    'price_move': [
        lambda timestep, df=df.copy(): float(df.iloc[timestep - 1]['price_move'])
        for df in debt_price_data
    ],
    'kp': kp_sweep,
    'ki': ki_sweep,
    # Select or sweep the error term calculation, as a lambda
    # e.g. p*-p vs (p*-p)/p vs (p*-p)/p*
    'error_term': [
        lambda target, measured: target - measured,
        #lambda target, measured: (target - measured) / measured,
        #lambda target, measured: (target - measured) / target
    ]
}

'''
The ConfigWrapper allows you to pass a model as an argument, and update the simulation configuration.
Maps (params, states) would be merge updated, and all other options are overrides.
'''
market_simulation = ConfigWrapper(market_model, M=update_params, N=1, T=SIMULATION_TIMESTEPS)

## Simulation Execution

In [11]:
from cadCAD import configs
del configs[:]

market_simulation.append()

(data, tensor_field, sessions) = run(drop_midsteps=True)


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (720, 21, 1, 15)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded


OverflowError: (34, 'Result too large')

In [ ]:
'''
To save state data to CSV:
'''
# compression_opts = dict(method='zip', archive_name='data.csv')  
# data.to_csv('data.zip', index=False, compression=compression_opts)

## Data Analysis

In [ ]:
df = data.copy()
df

In [ ]:
FXcols = [
    'error_star',
    'error_hat',
    'old_error_star',
    'old_error_hat',
    'error_star_integral',
    'error_hat_integral',
    'error_star_derivative',
    'error_hat_derivative',
    'target_rate',
    'target_price',
    'market_price',
    'debt_price'
]

In [ ]:
for c in FXcols:
    print(c)
    df[c] = df[c].apply(lambda x: float(x))

In [ ]:
df['target_rate_hourly'] = df.target_rate * 3600
df['error_star_derivative_scaled'] = df.error_star_derivative * 3600
df['error_hat_derivative_scaled'] = df.error_star_derivative * 3600

In [ ]:
import plotly.express as px

In [ ]:
fig = px.line(
    df,
    x='timestamp',
    y=['debt_price'],
    facet_col='subset',
    facet_col_wrap=5,
    #facet_row='run',
    template='seaborn',
    height=800
)

fig.show()

In [ ]:
fig = px.line(
    df,
    x='timestamp',
    y=['target_rate_hourly', 'error_star'],
    facet_col='subset',
    #facet_col='run',
    facet_col_wrap=3,
    template='seaborn',
    height=800
)

fig.show()

In [ ]:
fig = px.line(
    df,
    x='timestamp',
    y=['debt_price', 'target_price', 'market_price'],
    facet_col='subset',
    facet_col_wrap=4,
    #facet_col='run',
    template='seaborn',
    height=800
)

fig.for_each_annotation(lambda a: a.update(text=f'''price_move:{
    test_dfs[int(a.text.split("=")[-1])].price_move.max()
}'''))

fig.show()

In [ ]:
fig = px.line(
    df,
    x='timestamp',
    y=['error_star','error_star_derivative_scaled'],
    facet_col='subset',
    facet_col_wrap=4,
    #facet_col='run',
    template='seaborn',
    height=800
)

fig.show()

In [ ]:
from utils.plots import integral_plot

integral_plot(df[df.timestep<100])

Plotly `hist()` error, see: https://github.com/plotly/plotly.py/pull/2713

In [ ]:
df.timedelta.apply(lambda x: x/3600).hist()

In [ ]:
df.error_star.hist()

In [ ]:
import seaborn as sns; sns.set(style="white", color_codes=True)

In [ ]:
g = sns.jointplot(x="error_hat", y="error_star", alpha=.3, data=df[df.subset == 0])